<a href="https://colab.research.google.com/github/taizun-jj202/PyTorch_Basics/blob/NN/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import  torch.nn as nn
from torch.utils.data import DataLoader     # Loads data into training and test batches
import torchvision.datasets as datasets     # Gives the proper dataset from Pytorch
import torchvision.transforms as transfroms
import torch.nn.functional as F             # Contains activation functions
import  torch.optim as optim                # Contains optimizers


In [ ]:
# Defining Hyperparameters
in_channel = 1
batch_size = 64
num_classes = 10
epochs = 5
learning_rate = 0.001


In [ ]:
# Resource : https://learnopencv.com/number-of-parameters-and-tensor-sizes-in-convolutional-neural-network/#:~:text=Size%20of%20Output%20Tensor%20(Image)%20of%20a%20MaxPool%20Layer&text=%3D%20Pool%20size.&text=same%20as%20the%20kernel%20size,maxpool%20layer's%20output%20is%20unchanged.&text=So%2C%20the%20output%20image%20is%20of%20size%2027x27x96.
# Resource2: https://androidkt.com/calculate-output-size-convolutional-pooling-layers-cnn/


In [ ]:
# Defining the NN
class CNN(nn.Module):

    def __init__(self, num_classes = 10, in_channels = 1):
        super().__init__()
        self.conv1 = nn.Conv2d( in_channels=1, out_channels=16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        self.pool = nn.MaxPool2d( kernel_size=(1, 1), stride=(1, 1))
        self.conv2 = nn.Conv2d( in_channels=16, out_channels=32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        self.pool = nn.MaxPool2d( kernel_size=(1, 1), stride=(1, 1)) # Size of output 28*28*32
        self.fc1 = nn.Linear( in_features=28*28*32, out_features=100)
        self.fc2 = nn.Linear( in_features=100, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


In [ ]:

# Loading and splitting the data
train_ds = datasets.FashionMNIST( root= 'datasetFMNIST/',
                                  train= True,
                                  download=True,
                                  transform= transfroms.ToTensor())
train_load = DataLoader( dataset = train_ds,
                         batch_size= batch_size,
                         shuffle= True)

test_ds = datasets.FashionMNIST( root='datasetFMNIST/',
                                 transform=transfroms.ToTensor(),
                                 download=True,
                                 train=False)
test_load = DataLoader( dataset = test_ds,
                        batch_size= batch_size,
                        shuffle= True)


In [ ]:

# Validating the model:
def validate(loader, model):

    correct = 0
    tot_samples = 0

    with torch.no_grad():
        for (data, index) in loader:
            data = data.to(device)
            index = index.to(device)

            score = model(data)
            _, result = torch.max(data, 1)

            correct += (result == index).sum()
            tot_samples += result.size(0)       # Checks how many rows have passed i.e the no. of samples as 1row = 1 sample

        print(f"Accuracy : {float(correct) / float(tot_samples)}, Samples: {correct} / {tot_samples}")
    #model.train()

In [ ]:
def accuracy(loader, model):
    if loader.dataset.train :
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on testing data")

    num_correct = 0
    num_passes = 0

    with torch.no_grad():
        for (x, y) in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _ ,  pred = scores.max(1)
            num_correct += (pred == y).sum()
            num_passes += pred.size(0)
        print(f"Got {num_correct}/{num_passes} with accuracy {float(num_correct)/float(num_passes)*100}")

    model.train()

In [ ]:

# Training the model :
def train(loader,  model):

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),  lr = learning_rate)
    for epoch in range(epochs):
        for batch_idx, (data,index) in enumerate(loader):
            data = data.to(device)
            index = index.to(device)

            # Calculating loss function based on 'data' input
            loss = criterion(model(data), index)

            # Using optimizer to calculate lowest gradient
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


In [ ]:
# Instantiate the NN
device = 'cuda' if torch.cuda.is_available() else 'cpu'
FM_NN = CNN().to(device)
device

'cuda'

In [ ]:
# Output of the model
print("Training the model :")
train(train_load, FM_NN)


Training the model :


In [ ]:

print("Accuracy on train data :")
validate(train_load, FM_NN)
validate(test_load, FM_NN)


Accuracy on train data :


RuntimeError: ignored